<a href="https://colab.research.google.com/github/TapanManu/Duplicate-Questions-Pairs/blob/master/MLP/Duplicates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk

In [2]:
import pandas as pd
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [3]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import matplotlib.pyplot as plt

In [4]:
from google.colab import files
uploaded = files.upload()

Saving sample_qns.csv to sample_qns.csv


In [5]:
data=pd.read_csv('sample_qns.csv')

In [6]:
data.columns

Index(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate'], dtype='object')

In [7]:
data.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [8]:
lem = WordNetLemmatizer()
ps = PorterStemmer()

In [9]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
import numpy as np
stop_words = set(stopwords.words('english'))
def remove_punc(data):
  symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
  for i in symbols:
    data = np.char.replace(data, i, ' ')
  
def remove_apostrophe(data):
  return np.char.replace(data, "'", "")



In [11]:
data['question1']=data['question1'].apply(lambda x:x.lower())
data['question2']=data['question2'].apply(lambda x:x.lower())

In [12]:
data.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0
1,1,3,4,what is the story of kohinoor (koh-i-noor) dia...,what would happen if the indian government sto...,0
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0
3,3,7,8,why am i mentally very lonely? how can i solve...,find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"which one dissolve in water quikly sugar, salt...",which fish would survive in salt water?,0


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [15]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

Cleaning and preprocessing text

In [16]:
#cleaning and text pre processing
for j in ['question1','question2']:
   data[j]=data[j].str.replace('[^\w\s]','')
   data[j]=data[j].str.split(' ')
   data[j]=data[j].apply(lambda x:[lem.lemmatize(y) for y in x if y not in stop_words])
   data[j]=data[j].apply(lambda x:[ps.stem(y) for y in x])


In [17]:
data['question1']=[" ".join(q) for q in data['question1'].values]
data['question2']=[" ".join(q) for q in data['question2'].values]

In [18]:
def intersection(a, b):
    return ' '.join(list(set(a.split()) & set(b.split())))

def differencing(a, b):
    return ' '.join(list(set(a.split()) ^ set(b.split())))

In [19]:
data['interseq'] = data[['question1','question2']].apply(lambda x: intersection(*x), axis=1)
data['diffseq'] = data[['question1','question2']].apply(lambda x: differencing(*x), axis=1)    
data['q1d'] = data[['question1','diffseq']].apply(lambda x: intersection(*x), axis=1)    
data['q2d'] = data[['question2','diffseq']].apply(lambda x: intersection(*x), axis=1)

In [20]:
tf=TfidfVectorizer(ngram_range=(1,3))
questions = pd.Series(pd.concat([data['question1'],data['question2']]),dtype=str)
X_tfdf = tf.fit(questions)

In [21]:
X_q1_tfidf = tf.transform(data.question1)
X_q2_tfidf = tf.transform(data.question2)

In [22]:
questionI_tfidf = tf.transform(data.interseq)    
questionD_tfidf = tf.transform(data.diffseq) 
questionQ1D_tfidf = tf.transform(data.q1d)    
questionQ2D_tfidf = tf.transform(data.q2d)

In [23]:
print(X_q1_tfidf.shape)
print(X_q2_tfidf.shape)
print(questionI_tfidf.shape)
print(questionD_tfidf.shape)
print(questionQ1D_tfidf.shape)
print(questionQ2D_tfidf.shape)

(3966, 59019)
(3966, 59019)
(3966, 59019)
(3966, 59019)
(3966, 59019)
(3966, 59019)


In [70]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=1)
feat = pd.DataFrame(svd.fit_transform(X_q1_tfidf))
temp=pd.DataFrame(svd.fit_transform(X_q2_tfidf),columns=['1'])
feat = pd.concat([feat,temp],axis=1)
temp1=pd.DataFrame(svd.fit_transform(questionI_tfidf),columns=['2'])
temp2=pd.DataFrame(svd.fit_transform(questionD_tfidf),columns=['3'])
temp3=pd.DataFrame(svd.fit_transform(questionQ1D_tfidf),columns=['4'])
temp4=pd.DataFrame(svd.fit_transform(questionQ2D_tfidf),columns=['5'])
feat = pd.concat([feat,temp1,temp2,temp3,temp4],axis=1)
feat.fillna(0.0) #features only dataframe

,0,1,2,3,4,5
0,0.027422,0.014137,3.729771e-02,0.015504,0.000635,1.764415e-18
1,0.003370,0.009056,3.209104e-08,0.020461,0.000662,2.027396e-03
2,0.012946,0.010836,3.888139e-03,0.011901,0.003890,1.508475e-05
3,0.001313,0.003381,5.066754e-17,0.005746,0.000104,4.550667e-03
4,0.006911,0.007017,1.349252e-03,0.015739,0.004550,1.528918e-03
...,...,...,...,...,...,...
3961,0.010155,0.008383,1.090722e-02,0.008835,-0.000000,7.999161e-04
3962,0.001988,0.001318,-3.514861e-08,0.002495,0.000157,1.998106e-04
3963,0.006663,0.002772,-1.372306e-08,0.012011,0.000598,5.272902e-04
3964,0.009578,0.002348,6.532561e-03,0.010539,0.004087,3.660859e-05


In [52]:
from sklearn.metrics.pairwise import cosine_similarity
def get_cosine_similarity(feature_vec_1, feature_vec_2):    
    return cosine_similarity(feature_vec_1, feature_vec_2)

In [71]:
import numpy as np
X=feat

y = data['is_duplicate']

In [72]:
print(X.shape)
print(y.shape)

(3966, 6)
(3966,)


In [134]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score, roc_auc_score
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn.metrics import confusion_matrix

In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [75]:
from xgboost import XGBClassifier
xg = XGBClassifier()
xg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [76]:
pred_xgb = xg.predict_proba(X_test)[:,1]
logloss_xg = log_loss(y_test, pred_xgb)


logloss_xg

0.544584840220594

In [121]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Bidirectional,Dropout,BatchNormalization
from keras.utils.np_utils import to_categorical


In [130]:
model = Sequential()

model.add(BatchNormalization(input_shape=(6,)))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
model.add(BatchNormalization())
print(model.summary())


model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X_train,y_train,epochs=200,batch_size=40)

Model: "sequential_56"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_14 (Batc (None, 6)                 24        
_________________________________________________________________
dense_152 (Dense)            (None, 200)               1400      
_________________________________________________________________
dropout_29 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_153 (Dense)            (None, 200)               40200     
_________________________________________________________________
dropout_30 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_154 (Dense)            (None, 200)               40200     
_________________________________________________________________
dropout_31 (Dropout)         (None, 200)             

In [131]:
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

794/794 [==============================] - 0s 111us/step
Accuracy: 66.50


In [135]:
yhat_classes = model.predict_classes(X_test, verbose=0)
# precision tp / (tp + fp)
precision = precision_score(y_test, yhat_classes)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, yhat_classes)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, yhat_classes)
print('F1 score: %f' % f1)

Precision: 0.634783
Recall: 0.245791
F1 score: 0.354369


In [136]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X_train,y_train,epochs=300,batch_size=50)

Epoch 1/300
3172/3172 [==============================] - 1s 368us/step - loss: 0.6058 - accuracy: 0.6813
Epoch 2/300
3172/3172 [==============================] - 0s 121us/step - loss: 0.6065 - accuracy: 0.6731
Epoch 3/300
3172/3172 [==============================] - 0s 98us/step - loss: 0.6029 - accuracy: 0.6747
Epoch 4/300
3172/3172 [==============================] - 0s 91us/step - loss: 0.6054 - accuracy: 0.6810
Epoch 5/300
3172/3172 [==============================] - 0s 93us/step - loss: 0.6009 - accuracy: 0.6781
Epoch 6/300
3172/3172 [==============================] - 0s 93us/step - loss: 0.6075 - accuracy: 0.6879
Epoch 7/300
3172/3172 [==============================] - 0s 103us/step - loss: 0.6065 - accuracy: 0.6750
Epoch 8/300
3172/3172 [==============================] - 0s 90us/step - loss: 0.6024 - accuracy: 0.6835
Epoch 9/300
3172/3172 [==============================] - 0s 89us/step - loss: 0.6037 - accuracy: 0.6822
Epoch 10/300
3172/3172 [==============================] - 0s 

In [137]:
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))
yhat_classes = model.predict_classes(X_test, verbose=0)
# precision tp / (tp + fp)
precision = precision_score(y_test, yhat_classes)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, yhat_classes)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, yhat_classes)
print('F1 score: %f' % f1)

794/794 [==============================] - 0s 138us/step
Accuracy: 65.99
Precision: 0.619469
Recall: 0.235690
F1 score: 0.341463
